#### Import and load data

In [71]:
import pandas as pd
import math

In [2]:
df = pd.read_csv("Data_Exercise2.csv")
df.head()

,Salary,Working Time,Loan Decision
0,10,1.0,1
1,5,2.0,1
2,6,1.8,1
3,7,1.0,1
4,8,2.0,1


In [81]:
x_train = df.drop('Loan Decision', axis='columns').values.tolist()
y_train = df['Loan Decision'].tolist()

#### My Logistic Function

In [96]:
class LogisticRegression:
    def __init__(self):
        self.__feature_number = 0
        self.loss_ = 0
        self.grad_desc:list[float] = []
        self.weights:list[float] = []
        self.__y_hat:list[float] = []
        self.accuracy_ = 0

    def __init_value(self) -> list[float]:
        """Initialize a list of 0 that match the number of features"""
        new_ = []
        for i in range(self.__feature_number):
            new_.append(0)
        return new_

    def pad_one(self,x:list[list[float]]) -> list[list[float]]:
        """Add a column of 1"""
        x_ = []
        
        for i in range(len(x)):
            x_.append([])
            for j in range(len(x[0])):
                x_[i].append(x[i][j])
            x_[i].append(1)

        return x_

    def __sigmoid(self, x:list[float]) -> float:
        theta_x = 0

        for i in range(self.__feature_number):
            theta_x += self.weights[i]*x[i]
        
        return 1 / (1 + math.exp(-theta_x))

    def forward(self, x:list[float]):
        self.__y_hat = []
        
        for i in range(len(x)):
            self.__y_hat.append(self.__sigmoid(x[i]))
        
        return self.__y_hat

    def loss(self, x:list[float], Y:list[float]):
        m = len(x)
        l = 0
        
        for i in range(m):
            if Y[i] == 0:
                l += -math.log(1 - self.__y_hat[i], math.e)
            else:
                l += -math.log(self.__y_hat[i], math.e)
        
        self.loss_ = l/m

    def backward(self, x:list[list[float]], Y:list[float], lr:float):
        m = len(x)

        for i in range(m):
            for f in range(self.__feature_number):
                self.grad_desc[f] += (self.__y_hat[i] - Y[i]) * x[i][f]
        
        for i in range(self.__feature_number):
            self.weights[i] -= lr * self.grad_desc[i]/m

    def reset_grad(self):
        self.grad_desc = self.__init_value()

    def fit(self, x:list[float], Y:list[float], epochs=100, lr=0.01, pad_=True, print_log = False):
        if pad_:
            x = self.pad_one(x)

        self.__feature_number = len(x[0])
        self.weights = self.__init_value()
        self.grad_desc = self.__init_value()
        
        for epoch in range(epochs):
            self.forward(x)
            self.loss(x, Y)
            self.backward(x, Y, lr)
            self.reset_grad()
            
            if epoch%10 == 0 and print_log:
                print(f'{epoch}: {self.loss_}, {self.weights}')

    def predict(self, x:float|list[float]) -> float|list[float]:
        if isinstance(x, list):
            x = self.pad_one(x)
            return self.forward(x)
        
        raise TypeError
    
    def accuracy(self, x:list[float], Y:list[float]):
        try:
            yh = self.predict(x)
        except TypeError:
            print("Input must be a list, got a single value")
            return 
        
        true_pred = 0.0
        
        for i, value in enumerate(yh):
            if round(value) == Y[i]:
                true_pred += 1
        
        self.accuracy_ = round((true_pred / len(x))*100,2)

In [98]:
LR = LogisticRegression()

In [99]:
LR.fit(x_train, y_train, epochs=1000, lr=0.1, pad_=True, print_log=False)
LR.accuracy(x_train, y_train)
LR.accuracy_

95.0